In [10]:
# ============================================================
# CloudAudit — Automated Configuration & Compliance Auditor (Colab-ready)
# ============================================================

import os
import json
import random
from datetime import datetime, timezone

# Configuration
CONFIG_FILE = "data/configurations.json"
AUDIT_REPORT = "data/audit_report.json"

RULES = {
    "encryption": True,
    "min_tls": "1.2",
    "no_public": True
}

# Ensure directories
os.makedirs(os.path.dirname(CONFIG_FILE), exist_ok=True)
os.makedirs(os.path.dirname(AUDIT_REPORT), exist_ok=True)

# Generate dummy configurations if missing
if not os.path.exists(CONFIG_FILE):
    configs = []
    for i in range(1, 51):
        configs.append({
            "id": f"res-{i:03}",
            "type": random.choice(["storage_bucket", "compute_vm", "database"]),
            "encrypted": random.choice([True, False]),
            "tls_version": random.choice(["1.0", "1.1", "1.2", "1.3"]),
            "public_access": random.choice([True, False])
        })
    with open(CONFIG_FILE, "w", encoding="utf-8") as f:
        json.dump(configs, f, indent=2)

def load_configs(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def evaluate_one(cfg):
    issues = []
    if RULES["encryption"] and not cfg.get("encrypted", False):
        issues.append("encryption_disabled")
    if cfg.get("tls_version", "") < RULES["min_tls"]:
        issues.append("tls_too_low")
    if RULES["no_public"] and cfg.get("public_access", False):
        issues.append("public_access_allowed")
    return issues

def run_audit(input_path, output_path):
    configs = load_configs(input_path)
    report = {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "total": len(configs),
        "compliant": 0,
        "non_compliant": 0,
        "failures": []
    }
    for cfg in configs:
        issues = evaluate_one(cfg)
        resource_id = cfg.get("id", "<unknown_id>")
        if issues:
            report["non_compliant"] += 1
            report["failures"].append({"resource_id": resource_id, "issues": issues})
        else:
            report["compliant"] += 1
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)
    return report

if __name__ == "__main__":
    audit = run_audit(CONFIG_FILE, AUDIT_REPORT)
    print("✅ Audit complete:", audit)


✅ Audit complete: {'timestamp': '2025-10-19T10:22:56.279946+00:00', 'total': 30, 'compliant': 0, 'non_compliant': 30, 'failures': [{'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'public_access_allowed']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'tls_too_low']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'tls_too_low']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'tls_too_low']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'tls_too_low', 'public_access_allowed']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'public_access_allowed']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled']}, {'resource_id': '<unknown_id>', 'issues': ['encryption_disabled', 'tls_too_low']}, {'resource_id': '<unknown_id>', 'issues